##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Keras 구성요소 튜토리얼

***TensorFlow Extended(TFX)에 대한 구성요소별 소개***

참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행할 것을 권장합니다! "Google Colab에서 실행"을 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/components_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/components_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

이 Colab 기반 튜토리얼은 TensorFlow Extended(TFX)의 각 기본 제공 구성요소를 대화식으로 안내합니다.

여기서는 데이터 수집부터 모델 푸시, 제공에 이르기까지 종단 간 머신러닝 파이프라인의 모든 단계를 다룹니다.

완료되면 이 노트북의 콘텐츠를 TFX 파이프라인 소스 코드로 자동으로 내보낼 수 있으며, 이를 Apache Airflow 및 Apache Beam으로 오케스트레이션할 수 있습니다.

참고: 이 노트북은 TFX 파이프라인에서 기본 Keras 모델을 사용하는 방법을 보여줍니다. **TFX는 TensorFlow 2 버전의 Keras만 지원합니다**.

## 배경 설명

이 노트북은 Jupyter/Colab 환경에서 TFX를 사용하는 방법을 보여줍니다. 여기에서는 대화형 노트북에서 시카고 택시의 예를 살펴보겠습니다.

대화형 노트북에서 작업하면 TFX 파이프라인의 구조에 보다 쉽고 원활하게 익숙해질 수 있습니다. 또한 경량 개발 환경으로 자체 파이프라인을 개발할 때도 유용하지만 대화형 노트북이 오케스트레이션되는 방식과 메타데이터 아티팩트에 액세스하는 방식에 차이가 있음을 알고 있어야 합니다.

### 오케스트레이션

TFX의 프로덕션 배포에서는 Apache Airflow, Kubeflow Pipelines 또는 Apache Beam과 같은 오케스트레이터를 사용하여 TFX 구성요소의 미리 정의된 파이프라인 그래프를 오케스트레이션합니다. 대화형 노트북에서 노트북 자체가 오케스트레이터이며 노트북 셀을 실행할 때 각 TFX 구성요소가 실행됩니다.

### 메타데이터

TFX의 프로덕션 배포에서는 ML 메타데이터(MLMD) API를 통해 메타데이터에 액세스합니다. MLMD는 메타데이터 속성을 MySQL 또는 SQLite와 같은 데이터베이스에 저장하고 메타데이터 페이로드를 파일 시스템과 같은 영구 저장소에 저장합니다. 대화형 노트북에서 속성과 페이로드는 모두 Jupyter 노트북 또는 Colab 서버의 `/tmp` 디렉터리에 있는 단기 SQLite 데이터베이스에 저장됩니다.

## 설정

먼저, 필요한 패키지를 설치 및 가져오고, 경로를 설정하고, 데이터를 다운로드합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 꼭 Colab에서 실행되도록 해야 합니다. 물론 로컬 시스템도 별도로 업그레이드할 수 있습니다.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### TFX 설치

**참고: Google Colab에서는 패키지 업데이트로 인해 이 셀을 처음 실행할 때 런타임을 다시 시작해야 합니다(런타임 &gt; 런타임 다시 시작...).**

In [ ]:
!pip install -U tfx

## 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, 위의 셀을 처음 실행할 때 런타임을 다시 시작해야 합니다(런타임 &gt; 런타임 다시 시작...). 이는 Colab이 패키지를 로드하는 방식 때문입니다.

### 패키지 가져오기

표준 TFX 구성요소 클래스를 포함하여 필요한 패키지를 가져옵니다.

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

라이브러리 버전을 확인해 보겠습니다.

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### 파이프라인 경로 설정

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### 예제 데이터 다운로드

TFX 파이프라인에서 사용할 예제 데이터세트를 다운로드합니다.

우리가 사용하는 데이터세트는 시카고 시에서 제공한 [Taxi Trips 데이터세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)입니다. 이 데이터세트의 열은 다음과 같습니다.

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

이 데이터세트를 사용하여 여행의 `tips`를 예측하는 모델을 구축합니다.

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

CSV 파일을 간단히 살펴보세요.

In [ ]:
!head {_data_filepath}

*면책 조항: 이 사이트는 시카고 시 공식 웹사이트인 www.cityofchicago.org의 원본 소스로부터 수정된 데이터를 사용하는 애플리케이션을 제공합니다. 시카고 시는 이 사이트에서 제공하는 데이터의 내용, 정확성, 적시성 또는 완전성에 대해 어떠한 주장도 하지 않습니다. 이 사이트에서 제공하는 데이터는 언제든지 변경될 수 있습니다. 이 사이트에서 제공되는 데이터는 자신의 책임 하에 사용되는 것으로 간주됩니다.*

### InteractiveContext 생성하기

마지막으로, 이 노트북에서 TFX 구성요소를 대화식으로 실행할 수 있게 해주는 InteractiveContext를 생성합니다.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## 대화식으로 TFX 구성요소 실행하기

다음 셀에서 TFX 구성요소를 하나씩 생성하고, 각각을 실행하며, 출력 아티팩트를 시각화합니다.

### ExampleGen

`ExampleGen` 구성요소는 일반적으로 TFX 파이프라인의 시작 부분에 있으며, 다음 작업을 수행합니다.

1. 데이터를 훈련 및 평가 세트로 분할합니다(기본적으로 2/3 훈련 + 1/3 평가).
2. 데이터를 `tf.Example` 형식으로 변환합니다([여기](https://www.tensorflow.org/tutorials/load_data/tfrecord)에서 자세히 알아보기).
3. 다른 구성요소가 액세스할 수 있도록 데이터를 `_tfx_root` 디렉터리에 복사합니다.

`ExampleGen`은 데이터 소스에 대한 경로를 입력으로 사용합니다. 이 경우 다운로드한 CSV가 포함된 `_data_root`가 이에 해당합니다.

참고: 이 노트북에서는 구성요소를 하나씩 인스턴스화하고 `InteractiveContext.run()`으로 이를 실행할 수 있습니다. 이와 대조적으로, 프로덕션 환경에서는 `Pipeline`에서 오케스트레이터에 전달할 모든 구성요소를 미리 지정합니다([(TFX 파이프라인 빌드 가이드](https://www.tensorflow.org/tfx/guide/build_tfx_pipeline) 참조).

#### 캐시 활성화

노트북에서 `InteractiveContext`를 사용하여 파이프라인을 개발할 때 개별 구성요소가 출력을 캐시하는 시기를 제어할 수 있습니다. 구성요소가 생성한 이전 출력 아티팩트를 다시 사용하려면 `enable_cache`를 `True`로 설정합니다. 예를 들어 코드를 변경하는 경우, 구성요소의 출력 아티팩트를 다시 계산하려면 `enable_cache`를 `False`로 설정합니다.

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

`ExampleGen`의 출력 아티팩트를 살펴보겠습니다. 이 구성요소는 교육 예제와 평가 예제의 두 가지 아티팩트를 생성합니다.

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

또한 처음 세 가지 교육 예를 살펴볼 수도 있습니다.

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

이제 `ExampleGen`이 데이터 수집을 완료했으므로 다음 단계는 데이터 분석입니다.

### StatisticsGen

`StatisticsGen` 구성요소는 데이터 분석과 다운스트림 구성요소에서 사용하기 위해 데이터세트에 대한 통계를 계산합니다. [TensorFlow 데이터 유효성 검사](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

`StatisticsGen`은 `ExampleGen` 사용하여 방금 수집한 데이터세트를 입력으로 사용합니다.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

`StatisticsGen` 실행이 완료되면 출력된 통계를 시각화할 수 있습니다. 다양한 플롯을 사용해 보세요!

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

`SchemaGen` 구성요소는 데이터 통계를 기반으로 스키마를 생성합니다. (스키마는 데이터세트에 있는 특성의 예상 범위, 유형 및 속성을 정의합니다.) 또한 [TensorFlow 데이터 유효성 검사](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

참고: 생성된 스키마는 최선의 노력(best-effort) 방식이며 데이터의 기본 속성만 유추하려고 합니다. 필요에 따라 사용자가 이를 검토하고 수정해야 합니다.

`SchemaGen`은 `StatisticsGen`으로 생성한 통계를 입력으로 사용하고, 기본적으로 교육 분할을 확인합니다.

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

`SchemaGen` 실행이 완료되면 생성된 스키마를 테이블로 시각화할 수 있습니다.

In [ ]:
context.show(schema_gen.outputs['schema'])

데이터세트의 각 특성은 속성과 함께 스키마 테이블에 행으로 표시됩니다. 또한 스키마는 범주형 특성이 취하는 모든 값(해당 도메인으로 표시됨)을 포착합니다.

스키마에 대한 자세한 내용은 [SchemaGen 설명서](https://www.tensorflow.org/tfx/guide/schemagen)를 참조하세요.

### ExampleValidator

`ExampleValidator` 구성요소는 스키마에 정의된 기대치를 기반으로 데이터의 이상 여부를 감지합니다. 또한 [TensorFlow 데이터 유효성 검사](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

`ExampleValidator`는 `StatisticsGen`의 통계와 `SchemaGen`의 스키마를 입력으로 사용합니다.

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

`ExampleValidator` 실행이 끝나면 이상 여부를 테이블로 시각화할 수 있습니다.

In [ ]:
context.show(example_validator.outputs['anomalies'])

이상 여부 테이블에서 이상이 없음을 알 수 있습니다. 이것은 우리가 예상한 결과인데, 이것은 우리가 분석한 첫 번째 데이터세트이고 스키마가 여기에 맞게 조정되었기 때문입니다. 이 스키마를 검토해야 합니다. 예상치 못한 내용이 있으면 데이터의 이상을 의미합니다. 검토가 완료되면 스키마를 사용하여 향후 데이터를 보호할 수 있으며 여기에서 생성된 이상을 사용하여 모델 성능을 디버그하고 시간 경과에 따라 데이터가 어떻게 전개되는지 이해하며 데이터 오류를 식별할 수 있습니다.

### 변환

`Transform` 구성요소는 학습 및 제공 모두에 대해 특성 엔지니어링을 수행합니다. 여기에 [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) 라이브러리가 사용됩니다.

`Transform`은 `ExampleGen`의 데이터, `SchemaGen`의 스키마 및 사용자 정의 Transform 코드가 포함된 모듈을 입력으로 사용합니다.

아래에서 사용자 정의 Transform 코드의 예를 살펴보겠습니다(TensorFlow Transform API에 대한 소개는 [튜토리얼 참조](https://www.tensorflow.org/tfx/tutorials/transform/simple)). 먼저, 특성 엔지니어링을 위한 몇 가지 상수를 정의합니다.

참고: `%%writefile` 셀 매직은 셀의 내용을 디스크에 `.py`로 저장합니다. 이렇게 하면 `Transform` 구성요소가 코드를 모듈로 로드할 수 있습니다.


In [ ]:
_taxi_constants_module_file = 'taxi_constants.py'

In [ ]:
%%writefile {_taxi_constants_module_file}

NUMERICAL_FEATURES = ['trip_miles', 'fare', 'trip_seconds']

BUCKET_FEATURES = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]
# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

CATEGORICAL_NUMERICAL_FEATURES = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

CATEGORICAL_STRING_FEATURES = [
    'payment_type',
    'company',
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

다음으로, 원시 데이터를 입력으로 받고 모델이 학습할 수 있는 변환된 특성을 반환하는 `preprocessing_fn`을 작성합니다.

In [ ]:
_taxi_transform_module_file = 'taxi_transform.py'

In [ ]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_BUCKET_FEATURES = taxi_constants.BUCKET_FEATURES
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = taxi_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[taxi_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  for key in _BUCKET_FEATURES:
    outputs[taxi_constants.t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

  for key in _CATEGORICAL_NUMERICAL_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

  # Was this passenger a big tipper?
  taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

이제 이 특성 엔지니어링 코드를 `Transform` 구성요소에 전달하고 실행하여 데이터를 변환합니다.

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform, enable_cache=True)

`Transform`의 출력 아티팩트를 살펴보겠습니다. 이 구성요소는 두 가지 유형의 출력을 생성합니다.

- `transform_graph`는 전처리 작업을 수행할 수 있는 그래프입니다(이 그래프는 제공 및 평가 모델에 포함됨).
- `transformed_examples`는 전처리된 훈련 및 평가 데이터를 나타냅니다.

In [ ]:
transform.outputs

`transform_graph` 아티팩트를 살펴보겠습니다. 세 개의 하위 디렉터리가 포함된 디렉터리를 가리킵니다.

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

`transformed_metadata` 하위 디렉터리에는 전처리된 데이터의 스키마가 포함됩니다. `transform_fn` 하위 디렉터리에는 실제 사전 처리 그래프가 포함되어 있습니다. `metadata` 하위 디렉터리에는 원본 데이터의 스키마가 포함됩니다.

변환된 처음 세 가지 예도 살펴볼 수 있습니다.

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

`Transform` 구성요소가 데이터를 특성으로 변환한 후 다음 단계는 모델을 훈련시키는 것입니다.

### 트레이너

`Trainer` 구성요소는 TensorFlow에서 정의한 모델을 훈련합니다. 기본 트레이너는 Estimator API를 지원하며 Keras API를 사용하려면 트레이너의 생성자에서 <code>custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)</code> 설정을 통해 <a>일반 트레이너</a>를 지정해야 합니다.

`Trainer`는 `SchemaGen`의 스키마, `Transform`의 변환된 데이터와 그래프, 훈련 파라미터 및 사용자 정의 모델 코드가 포함된 모듈을 입력으로 사용합니다.

아래에서 사용자 정의 모델 코드의 예를 살펴보겠습니다(TensorFlow Keras API에 대한 소개는 [튜토리얼 참조](https://www.tensorflow.org/guide/keras)).

In [ ]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [ ]:
%%writefile {_taxi_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_LABEL_KEY = taxi_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)
  
  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

이제 이 모델 코드를 `Trainer` 구성요소에 전달하고 실행하여 모델을 훈련합니다.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

#### TensorBoard로 훈련 분석하기

트레이너 아티팩트를 살펴보세요. 모델 하위 디렉터리가 포함된 디렉터리를 가리킵니다.

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

선택적으로 TensorBoard를 트레이너에 연결하여 모델의 훈련 곡선을 분석할 수 있습니다.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

### 평가자

`Evaluator` 구성요소는 평가 세트에 대한 모델 성능 메트릭을 계산합니다. 여기에 [TensorFlow 모델 분석](https://www.tensorflow.org/tfx/model_analysis/get_started) 라이브러리가 사용됩니다. `Evaluator`는 새로 훈련된 모델이 이전 모델보다 우수한지 선택적으로 검증할 수도 있습니다. 이는 매일 자동으로 모델을 훈련하고 검증할 수 있는 프로덕션 파이프라인 환경에서 유용합니다. 이 노트북에서는 하나의 모델만 훈련하므로 `Evaluator` 자동으로 모델에 "양호" 레이블을 지정합니다.

`Evaluator`는 `ExampleGen`의 데이터, `Trainer`의 훈련된 모델 및 슬라이싱 구성을 입력으로 사용합니다. 슬라이싱 구성을 사용하면 특성 값에 대한 메트릭을 분할할 수 있습니다(예: 오전 8시에 시작하는 택시 여행과 오후 8시에 시작하는 택시 여행에서 모델의 성능은 어떻습니까?). 아래에서 이 구성의 예를 참조하세요.

In [ ]:
# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=taxi_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(
            feature_keys=['trip_start_hour'])
    ])

다음으로 이 구성을 `Evaluator`에 제공하고 실행합니다.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

이제 `Evaluator`의 출력 아티팩트를 살펴보겠습니다.

In [ ]:
evaluator.outputs

`evaluation` 출력을 사용하여 전체 평가 세트에 대한 글로벌 메트릭의 기본 시각화를 표시할 수 있습니다.

In [ ]:
context.show(evaluator.outputs['evaluation'])

슬라이싱된 평가 메트릭에 대한 시각화를 보려면 TensorFlow 모델 분석 라이브러리를 직접 호출할 수 있습니다.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

이 시각화는 동일한 메트릭을 보여주지만 전체 평가 세트 대신 `trip_start_hour`의 모든 특성 값에서 계산됩니다.

TensorFlow 모델 분석은 공정성 지표 및 모델 성능의 시계열 플로팅과 같은 다른 많은 시각화를 지원합니다. 자세한 내용은 [튜토리얼](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic)을 참조하세요.

구성에 임계값을 추가했으므로 검증 출력도 사용할 수 있습니다. `blessing` 아티팩트의 존재는 우리 모델이 검증을 통과했음을 나타냅니다. 이것이 수행되는 첫 번째 검증이기 때문에 후보자는 자동으로 좋게 평가됩니다.

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

이제 검증 결과 기록을 로드하여 성공을 확인할 수도 있습니다.

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### 푸셔

`Pusher` 구성요소는 일반적으로 TFX 파이프라인 끝에 있습니다. 모델이 검증을 통과했는지 여부를 확인하고 통과한 경우 모델을 `_serving_model_dir`로 내보냅니다.

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

`Pusher`의 출력 아티팩트를 살펴보겠습니다.

In [ ]:
pusher.outputs

특히, 푸셔는 다음과 같은 SavedModel 형식으로 모델을 내보냅니다.

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

내장 TFX 구성요소 둘러보기를 마쳤습니다!